In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import gzip
import csv
import pandas as pd
import numpy as np
import re
import glob
from collections import defaultdict
path='/content/drive/My Drive/Courses/Advanced topic/project/Dataset/'
df_dna=pd.read_csv(path+"dna_mythylation_with_common_samples_with_dna_m.csv",index_col=0).sort_index()
df_gene=pd.read_csv("/content/drive/My Drive/Courses/Advanced topic/project/Dataset//Gene Expression/new_gene_expression_with_common_samples_with_dna_m.csv").sort_index()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import seaborn as sn
import matplotlib.pyplot as plt
# from keras.wrappers.scikit_learn import KerasClassifier
# from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import explained_variance_score
import tensorflow as tf
from tensorflow import keras 
from keras.models import Sequential, Model, Input
from keras.layers import Dense, Dropout
from sklearn import preprocessing
from tensorflow.keras.optimizers import SGD
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.feature_selection import VarianceThreshold
from pandas.io.formats.format import CategoricalFormatter
from sklearn.metrics import confusion_matrix
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [ ]:
df_pdi=pd.read_csv(path+'PPI/PPI_weights_new_gene_common_samples.csv',index_col=0)
df_pdi= df_pdi.sort_values(by=['Unnamed: 0.1'])
index = df_pdi['Unnamed: 0.1']
df_pdi= df_pdi.drop(['Unnamed: 0.1'],axis=1)
df_pdi.index =index
df_pdi

,CDH17,CEACAM5,HBEGF,KRAS,CBLB,ERBB3,RPS27A,RHOC,HSP90B1,PDGFB,...,RMND5B,OCIAD2,CNTD1,PGRMC2,TMEM206,LMO3,FAR2,TMEM14C,SMTN,MUC13
Unnamed: 0.1,,,,,,,,,,,,,,,,,,,,,
A1BG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1CF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ABCC3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ABHD13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ABHD17A,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZNF710,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ZNHIT1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ZSCAN16-AS1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
weights_pdi=[]
for i,cols in enumerate(df_pdi.columns):
  weights_pdi.append(df_pdi[cols].values)
weights_pdi=np.array(weights_pdi)

In [ ]:
if(np.all(df_dna.index == df_gene['samples'])):
  print("same")

same


In [ ]:
le = preprocessing.LabelEncoder()
scaler = StandardScaler()
print(len(df_gene['samples']))
print(len(df_gene))

data_gene=df_gene.drop(['samples',	'sample.name',	'true.labels$true.labels'],axis=1)
# data=df_trans.
data_gene=data_gene.iloc[:,:]
data_gene=(np.array(data_gene).astype(np.float32))
scaler.fit(data_gene)
data_gene=scaler.transform(data_gene)

data_dna=df_dna.drop(['regions','samples.1'],axis=1)
# data=df_trans.
data_dna=data_dna.iloc[:,:]
data_dna=(np.array(data_dna).astype(np.float32))
scaler.fit(data_dna)
data_dna=scaler.transform(data_dna)


609
609


In [ ]:
le = preprocessing.LabelEncoder()
le.fit(df_gene['true.labels$true.labels'])
y=le.transform(df_gene['true.labels$true.labels'])
le.fit(df_dna['regions'])
y2=le.transform(df_dna['regions'])

In [ ]:
def create_model():
  inputA = Input(shape=(data_gene.shape[1],))
  inputB = Input(shape=(data_dna.shape[1],))
  x= Dense(weights_pdi.shape[0],kernel_initializer='lecun_uniform', activation='sigmoid',name='set')(inputA)
  x= Dropout(0.2)(x)
  x = Dense(100, activation="relu",kernel_initializer='lecun_uniform')(x)
  x = Model(inputs=inputA, outputs=x)
 
  # the second branch opreates on the second input
  y = Dense(50, activation="sigmoid",kernel_initializer='lecun_uniform')(inputB)
  y= Dropout(0.2)(y)
  # y = Dense(4, activation="relu")(y)
  y = Model(inputs=inputB, outputs=y)
  combined = concatenate([x.output, y.output])
  # apply a FC layer and then a regression prediction on the
  # combined outputs
  z = Dense(12, activation="sigmoid")(combined)
  z= Dropout(0.1)(z)
  # z = Dense(10, activation="relu")(z)
  z = Dense(4, activation="sigmoid")(z)

  # our model will accept the inputs of the two branches and
  # then output a single value
  model = Model(inputs=[x.input, y.input], outputs=z)
  print(x.summary())
  x.layers[1].set_weights([weights_pdi.T, np.zeros(x.layers[1].get_weights()[1].shape)])
  optimizer = SGD(learning_rate=0.1, momentum=0.03)
  model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy',metrics= 'accuracy')
  return model

In [ ]:
def stratfiedk_fold_cross_validation(X,X2,y,y2,n,r):
  skf = StratifiedKFold(n_splits=n, shuffle=True, random_state=r)
  reports=[]
  test_score=[]
  train_score=[]
  best_score=-1
  cf=[]
  best_model=create_model()
  for train, test in skf.split(X,y):
    # print(train,test)
    Xtrain, X_val =X[train], X[test]
    Xtrain2, X_val2 =X2[train], X2[test]
    ytrain, y_val =y[train], y[test]
    ytrain2, y_val2 =y2[train], y2[test]
    print(len(Xtrain2),len(Xtrain))
    model=create_model()
    # model.fit(X_train,y_train,validation_data=(X_val,y_val),batch_size=80,epochs=50)
    model.fit(x=[Xtrain, Xtrain2], y=ytrain,validation_data=([X_val, X_val2], y_val),	epochs=50)

    y_pred=model.predict([X_val,X_val2])
    max_value = np.argmax(y_pred, axis=1)
    # max_index = y_pred.index(max_value)
    # print(max_value,y_test)
    if(best_score<f1_score(y_val, max_value, average='macro')):
      best_score=f1_score(y_val, max_value, average='macro')
      best_model=model
  
  return best_model

In [ ]:
from keras.layers import Dense, Dropout, concatenate
RANDOME_STATES=[100,2000,1000]
f1_folds_scores=[]
for r in RANDOME_STATES:
  skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=r)
  for train, test in skf.split(data_gene,y):
    X_train, X_test =data_gene[train], data_gene[test]
    X_train2, X_test2 = data_dna[train], data_dna[test]
    y_train, y_test =y[train], y[test]
    y_train2, y_test2 =y2[train], y2[test]
    best_model=stratfiedk_fold_cross_validation(X_train,X_train2,y_train,y_train2,5,r)
    y_pred=best_model.predict([X_test,X_test2])
    max_value = np.argmax(y_pred, axis=1)
    f1_folds_scores.append(classification_report(y_test, max_value, output_dict=True ))


Streaming output truncated to the last 5000 lines.
13/13 [==============================] - 1s 43ms/step - loss: 1.0353 - accuracy: 0.5231 - val_loss: 1.0422 - val_accuracy: 0.6186
Epoch 4/50
13/13 [==============================] - 1s 41ms/step - loss: 0.9633 - accuracy: 0.5949 - val_loss: 0.9488 - val_accuracy: 0.4639
Epoch 5/50
13/13 [==============================] - 1s 40ms/step - loss: 0.8895 - accuracy: 0.6359 - val_loss: 0.9123 - val_accuracy: 0.6495
Epoch 6/50
13/13 [==============================] - 1s 42ms/step - loss: 0.8589 - accuracy: 0.6513 - val_loss: 0.8515 - val_accuracy: 0.7320
Epoch 7/50
13/13 [==============================] - 1s 39ms/step - loss: 0.7711 - accuracy: 0.7564 - val_loss: 0.8172 - val_accuracy: 0.6186
Epoch 8/50
13/13 [==============================] - 1s 47ms/step - loss: 0.7119 - accuracy: 0.7923 - val_loss: 0.7977 - val_accuracy: 0.6289
Epoch 9/50
13/13 [==============================] - 1s 45ms/step - loss: 0.6597 - accuracy: 0.7872 - val_loss: 0.7

In [ ]:
#  f1_folds_scores[0]['0']
classes=['0','1','2','3']
precisions, recalls, f1_scores,supports=[],[],[],[]
accurcies,m_precision,m_recall,m_f1_score,w_precision,w_recall,w_f1_score=[],[],[],[],[],[],[]
for l in range(0,len(f1_folds_scores)):
  print(f1_folds_scores[l])
  for c in classes:
    precisions.append(f1_folds_scores[l][c]['precision'])
    recalls.append(f1_folds_scores[l][c]['recall'])
    f1_scores.append(f1_folds_scores[l][c]['f1-score'])
    supports.append(f1_folds_scores[l][c]['support'])
  accurcies.append(f1_folds_scores[l]['accuracy'])
  m_precision.append(f1_folds_scores[l]['macro avg']['precision'])
  m_recall.append(f1_folds_scores[l]['macro avg']['recall'])
  m_f1_score.append(f1_folds_scores[l]['macro avg']['f1-score'])
  w_precision.append(f1_folds_scores[l]['weighted avg']['precision'])
  w_recall.append(f1_folds_scores[l]['weighted avg']['recall'])
  w_f1_score.append(f1_folds_scores[l]['weighted avg']['f1-score'])

{'0': {'precision': 0.8214285714285714, 'recall': 1.0, 'f1-score': 0.9019607843137255, 'support': 23}, '1': {'precision': 1.0, 'recall': 0.9433962264150944, 'f1-score': 0.970873786407767, 'support': 53}, '2': {'precision': 0.9333333333333333, 'recall': 0.9032258064516129, 'f1-score': 0.9180327868852459, 'support': 31}, '3': {'precision': 1.0, 'recall': 0.9333333333333333, 'f1-score': 0.9655172413793104, 'support': 15}, 'accuracy': 0.9426229508196722, 'macro avg': {'precision': 0.9386904761904762, 'recall': 0.9449888415500101, 'f1-score': 0.9390961497465122, 'support': 122}, 'weighted avg': {'precision': 0.9493950039032006, 'recall': 0.9426229508196722, 'f1-score': 0.9437965879750788, 'support': 122}}
{'0': {'precision': 0.8148148148148148, 'recall': 0.9565217391304348, 'f1-score': 0.8800000000000001, 'support': 23}, '1': {'precision': 0.9423076923076923, 'recall': 0.9245283018867925, 'f1-score': 0.9333333333333333, 'support': 53}, '2': {'precision': 0.9259259259259259, 'recall': 0.8333

In [ ]:
results_classes= pd.DataFrame(list(zip(precisions,recalls,f1_scores,supports)),columns =['precision','recall','f1-score','support'])
results_classes

,precision,recall,f1-score,support
0,0.821429,1.000000,0.901961,23
1,1.000000,0.943396,0.970874,53
2,0.933333,0.903226,0.918033,31
3,1.000000,0.933333,0.965517,15
4,0.814815,0.956522,0.880000,23
5,0.942308,0.924528,0.933333,53
6,0.925926,0.833333,0.877193,30
7,1.000000,1.000000,1.000000,16
8,0.785714,0.956522,0.862745,23
9,0.959184,0.886792,0.921569,53


In [ ]:
overall_results= pd.DataFrame(list(zip(accurcies,m_precision,m_recall,m_f1_score,w_precision,w_recall,w_f1_score)),columns =['accuracy','macro_precision','macro_recall','macro_f1','weighted_precision','weighted_recall','weighted_f1_score'])
overall_results.describe()

,accuracy,macro_precision,macro_recall,macro_f1,weighted_precision,weighted_recall,weighted_f1_score
count,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000
mean,0.895457,0.887734,0.901559,0.890561,0.902802,0.895457,0.895723
std,0.026208,0.031409,0.027963,0.032134,0.023979,0.026208,0.027392
min,0.844262,0.833646,0.842249,0.818954,0.862288,0.844262,0.837581
25%,0.880639,0.867135,0.888511,0.873827,0.887552,0.880639,0.879908
50%,0.892562,0.883148,0.896245,0.885167,0.896642,0.892562,0.893322
75%,0.917694,0.911647,0.923161,0.914058,0.921772,0.917694,0.918177
max,0.942623,0.938690,0.944989,0.939096,0.949395,0.942623,0.943797


In [ ]:
save=True
if(save):
  results_classes.to_csv("/content/drive/My Drive/Courses/Advanced topic/project/CV_results/new_gene_expression_dna_mythelation_genebody_PPI_common_samples_CV_classwise_final.csv")
  overall_results.to_csv("/content/drive/My Drive/Courses/Advanced topic/project/CV_results/new_gene_expression_dna_mythelation_genebody_PPI_common_samples_CV_overall_final.csv")